In [1]:
pip install twilio

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
    --------------------------------------- 0.0/1.8 MB 325.1 kB/s eta 0:00:06
    --------------------------------------- 0.0/1.8 MB 326.8 kB/s eta 0:00:06
   -- ------------------------------------- 0.1/1.8 MB 476.3 kB/s eta 0:00:04
   --- ------------------------------------ 0.2/1.8 MB 807.1 kB/s eta 0:00:03
   -------- ------------------------------- 0.4/1.8 MB 1.5 MB/s eta 0:00:01
   ----------------- ---------------------- 0.8/1.8 MB 2.7 MB/s eta 0:00:01
   ----------------------------- ---------- 1.3/1.8 MB 3.9 MB/s eta 0:00:01
   ------------------------------------- -- 1.7/1.8 MB 4.4 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 4.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib
from twilio.rest import Client
from twilio.twiml.voice_response import VoiceResponse

data = pd.read_csv('synthetic_health_data.csv')

# Features and target
X = data[['pulse_rate', 'body_temp', 'ambient_noise', 'movement', 'stress_indicator', 'user_behavior', 'spo2']]
y = data['danger']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Test accuracy
y_pred = rf_classifier.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

# Save the model
joblib.dump(rf_classifier, 'danger_classifier_model.pkl')

# Function to make an automated call if danger is detected
def make_automated_call():
    # Twilio account credentials (replace with your actual credentials)
    account_sid = 'SID'
    auth_token = 'TOKEN'
    twilio_number = '+17722526488'
    to_phone_number = '+919347768901'  # replace with the number you want to call

    # Create a Twilio client
    client = Client(account_sid, auth_token)

    # Create a TwiML response
    response = VoiceResponse()
    response.say("you are child is in danger . Please take necessary actions. you are child is in danger . Please take necessary actions. ", voice='alice')

    # Make the call
    call = client.calls.create(
        to=to_phone_number,
        from_=twilio_number,
        twiml=response  # Use the TwiML directly here
    )
    print(f"Call initiated: {call.sid}")

# Predict on new input data (for testing the automated call system)
def check_for_danger(new_input):
    # Load the saved model
    rf_classifier = joblib.load('danger_classifier_model.pkl')
    
    prediction = rf_classifier.predict([new_input])
    
    if prediction[0] == 1:
        print("Danger detected! Initiating automated call...")
        make_automated_call()
    else:
        print("No danger detected.")

# Example new input for prediction
new_input = [120, 102.5, 65, 1, 0.8, 1, 88]  # sample input corresponding to a 'danger' situation
check_for_danger(new_input)


Accuracy: 0.95
Danger detected! Initiating automated call...


C:\Users\chetan\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Call initiated: CA62439357d9d1e1040545256d6facfd2d
